In [10]:
import torch
import torchvision.transforms as transforms               # include image preprocess tools
from torchvision.datasets import CIFAR100        # for loading images from Pytorch CIFAR
from torch.utils.data import DataLoader
import detectors
import timm
from src.temperature_scaling import ModelWithTemperature
from src.raps_hyp_opt import lambda_optimization_raps, k_reg_optimization

# check GPU status
print("Is CUDA available:", torch.cuda.is_available())
print("Device count:", torch.cuda.device_count())
print("Device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# load fine-tuned model
model = timm.create_model("resnet50_cifar100", pretrained=True)
model = model.to(device)

# reprocess the images from CIFAR
data_transform = transforms.Compose([
    transforms.ToTensor(),          # transfer to tensor
    transforms.Normalize((0.5071, 0.4865, 0.4409), (0.2673, 0.2564, 0.2762))  # normalize
])
# load images from CIFAR10
dataset = CIFAR100(root="../../../data", train=False, download=True, transform=data_transform)

# tempreture scaling
temp_scal_loader = DataLoader(dataset, batch_size=32, shuffle=True)
model = ModelWithTemperature(model, temperature = 4.85).to(device)
model.set_temperature(temp_scal_loader)

model.eval()

lambda_values = [0, 1e-4, 1e-3, 0.01, 0.02, 0.05, 0.1, 0.2, 0.5, 0.7, 1.0]
k_reg_values = [1, 2, 3, 5, 7, 9, 10]

# lambda optimization
print("Looking for optimal lambda...")
optimal_lambda = lambda_optimization_raps(model, dataset, lambda_values, k_reg = 5, device=device)
if optimal_lambda == None:
    print("No optimal lambda is found")
else:
    print(f"Optimal lambda is {optimal_lambda}\n")
    
    # k_reg optimization
    print("Looking for optimal k_reg...")
    optimal_k = k_reg_optimization(model, dataset, optimal_lambda, k_reg_values, device=device)
    
    if optimal_k == None:
        print("No optimal k_reg is found")
    else:
        print(f"Optimal k_reg is {optimal_k}")

Is CUDA available: True
Device count: 1
Device name: NVIDIA GeForce RTX 3060 Ti
Files already downloaded and verified
Before temperature - NLL: 1.072, ECE: 0.083
Optimal temperature: 4.692
After temperature - NLL: 2.955, ECE: 0.674
Looking for optimal lambda...
Optimal lambda is 0.5

Looking for optimal k_reg...
Optimal k_reg is 2


# 8. Result

- Optimal lambda = **0.5**
- Optimal k_reg = **2**